# Semantic Archetype Selection Prototype

This notebook prototypes a new way to select Pneuma's archetypes using **Semantic Similarity** (Embeddings) instead of keyword matching.

**The Goal:**
Instead of triggering "Mystic" only when someone says "God" or "Spirit", we want to trigger it when the *vibe* of the message is mystical, even if the words are different.

**The Method:**
1. Define "Essence Vectors" for each archetype (a description of what they represent).
2. Embed the user's message.
3. Find the closest archetype vector.


In [1]:
# 1. Setup
import os
from openai import OpenAI
import numpy as np
from dotenv import load_dotenv

# Load API key from server/.env
load_dotenv("server/.env")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

print("OpenAI Client Initialized.")


OpenAI Client Initialized.


In [2]:
# 2. Define Archetype Essences
# These descriptions define the "center of gravity" for each archetype.

archetype_essences = {
    "mystic": "Direct experience of the divine, silence, paradox, non-duality, the ineffable, spiritual presence, ego death.",
    "darkScholar": "Existential dread, suffering as truth, pessimism, the void, harsh reality, unblinking observation of darkness.",
    "trickster": "Humor, subversion, breaking rules, absurdity, satire, mocking authority, playfulness, chaos.",
    "warriorSage": "Discipline, strategy, clarity, action, stillness in motion, mastery, economy of force.",
    "prophetPoet": "Beauty, longing, emotional depth, love, tenderness, the heart's wisdom, lyrical expression.",
    "inventor": "Curiosity, structure, how things work, engineering, systems thinking, design, observation.",
    "antifragilist": "Risk, uncertainty, skin in the game, resilience, thriving in chaos, skepticism of experts.",
    "stoicEmperor": "Duty, acceptance, control over self, rationality, endurance, calm amidst storm."
}

print(f"Defined {len(archetype_essences)} archetype essences.")


Defined 8 archetype essences.


In [3]:
# 3. Pre-calculate Archetype Embeddings
# In production, we would do this once on startup.

archetype_vectors = {}
print("Generating archetype vectors...")

for name, description in archetype_essences.items():
    vector = get_embedding(description)
    archetype_vectors[name] = vector
    print(f"Encoded {name}")

print("Done.")


Generating archetype vectors...
Encoded mystic
Encoded darkScholar
Encoded trickster
Encoded warriorSage
Encoded prophetPoet
Encoded inventor
Encoded antifragilist
Encoded stoicEmperor
Done.


In [4]:
# 4. Test the Semantic Router

def find_best_archetype(user_message):
    message_vector = get_embedding(user_message)
    
    scores = []
    for name, vector in archetype_vectors.items():
        score = cosine_similarity(message_vector, vector)
        scores.append((name, score))
    
    # Sort by score descending
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores

# Test Cases
test_messages = [
    "I feel like everything is pointless and full of suffering.",
    "Tell me a joke about the government.",
    "How do I build a better mousetrap?",
    "I want to feel the presence of God.",
    "I need to toughen up and face my problems.",
    "Why is beauty so painful?"
]

print("--- RESULTS ---")
for msg in test_messages:
    results = find_best_archetype(msg)
    top_match = results[0]
    print(f"\nMessage: '{msg}'")
    print(f"Top Match: {top_match[0].upper()} ({top_match[1]:.4f})")
    print(f"Runner Up: {results[1][0]} ({results[1][1]:.4f})")


--- RESULTS ---

Message: 'I feel like everything is pointless and full of suffering.'
Top Match: DARKSCHOLAR (0.4337)
Runner Up: mystic (0.2249)

Message: 'Tell me a joke about the government.'
Top Match: TRICKSTER (0.2843)
Runner Up: inventor (0.1587)

Message: 'How do I build a better mousetrap?'
Top Match: INVENTOR (0.2402)
Runner Up: warriorSage (0.1979)

Message: 'I want to feel the presence of God.'
Top Match: MYSTIC (0.4539)
Runner Up: prophetPoet (0.2010)

Message: 'I need to toughen up and face my problems.'
Top Match: STOICEMPEROR (0.3333)
Runner Up: darkScholar (0.2594)

Message: 'Why is beauty so painful?'
Top Match: PROPHETPOET (0.4500)
Runner Up: darkScholar (0.3263)
